In [1]:
# 测试环境 :
#     appKey：bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6
#     secretKey ：4ed8b056c32939b9fd66987470b3e9fb720bdded02197e678e516bdcdf810833
#     渠道id：1200
#     测试环境地址：http://test1.thirdpart.kaolatest.netease.com/api/接口名称
# 接口文档地址：https://github.com/kaola-thirdpart/netease-kaola-thirdpart/wiki

In [12]:
import requests
import time
import json
import os
import urllib.parse
import hashlib
# 系统参数：

# 名称	类型	是否必须	描述
# timestamp	string	Y	时间戳，格式为yyyy-mm-dd HH:mm:ss，例如：2013-05-06 13:52:03。考拉API服务端允许客户端请求时间误差为6分钟。
# app_key	string	Y	分配给应用的AppKey ，创建应用时可获得
# v	string	Y	API协议版本，可选值:1.0。
# sign	string	Y	对 API 调用参数（除sign外）进行 md5 加密获得。获取方法参考如下1.4.1
# sign_method	string	Y	参数的加密方法选择，可选值是：md5

# test
# url = 'http://test1.thirdpart.kaolatest.netease.com/api/'
# channel_id = '1200'
# app_key = 'bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6'
# secret_key = '4ed8b056c32939b9fd66987470b3e9fb720bdded02197e678e516bdcdf810833'

channel_id = '35195'
app_key = '1912cc049d1f66b2daf49451dfb351b48cf7ba9d72bf591fc1f294c451c94450'
secret_key = '5295226c17645d477c5f19bebd947c9949eb9e8c37595abf6d1a6081b8a59d37'
url = 'http://thirdpart.kaola.com/api/'

def concat_params(params):
    pairs = []
    for key in sorted(params):
        if key == 'sign':
            continue
        val = params[key]
        pairs.append("{}{}".format(key, val))
    return ''.join(pairs)
def gen_sign(params, consumer_secret):
    # 根据参数名称（除签名和图片）将所有请求参数按照字母先后顺序排序:key + value .... key + value
    # 例如：将foo=1,bar=2,baz=3 排序为bar=2,baz=3,foo=1，参数名和参数值链接后，得到拼装字符串bar2baz3foo1
    # 系统同时支持MD5加密方式: md5：将secretKey 拼接到参数字符串头、尾进行md5加密后，再转化成大写，
    # 格式是：byte2hex(md5(secretKeykey1value1key2value2...secretKey)) 
    # 注：hex为自定义方法，JAVA中MD5是对字节数组加密，加密结果是16字节，我们需要的是32位的大写字符串
    params = concat_params(params)
    to_hash = hashlib.md5(bytes('{consumer_secret}{params}{consumer_secret}'.format(params=params,consumer_secret=consumer_secret),'utf-8'))
    sign = to_hash.hexdigest()
    return sign.upper()

In [13]:
def query_order_status(data):
    """渠道订单状态查询，接口地址：queryOrderStatus"""
    # channelId	String	是	渠道商ID	
    # timestamp	String	是	时间戳	
    # v	String	是	Api版本	
    # sign_method	String	是	签名方法	
    # app_key	String	是	考拉分发的appKey	
    # sign	String	是	签名	
    # thirdPartOrderId	String	是	分销商订单ID
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "thirdPartOrderId":data['orderId']
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryOrderStatus',params=body)
    return resp.json()
query_order_status({"orderId":"me004232688699697"})

{'recCode': -200, 'recMeg': '获取订单信息异常'}

In [17]:
def query_all_goods_id():
    """获取商品skuid列表，查询自己渠道下所有的商品skuid列表。
    接口说明：
    渠道下所有商品id查询，接口地址：queryAllGoodsId"""
    
    # channelId	String	是	渠道商ID	
    # timestamp	String	是	时间戳	
    # v	String	是	Api版本	
    # sign_method	String	是	签名方法	
    # app_key	String	是	考拉分发的appKey	
    # sign	String	是	签名
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryAllGoodsId',params=body)
    return resp.json()
query_all_goods_id()

{'goodsInfo': ['00000020130',
  '00000020140',
  '00000022000',
  '00000224270',
  '00000224270',
  '00000224270',
  '00000224270',
  '00000224270',
  '00000224280',
  '00000224280',
  '00000224280',
  '00000224280',
  '00000224280',
  '00000224290',
  '00000224290',
  '00000224290',
  '00000224290',
  '00000224290',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000282461',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000284391',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000298761',
  '00000467120',
  '00000467120',
  '00000467120',
  '00000467120',
  '00000467130',
  '00000467130',
  '00000467130',
  '00000467130',
  '00000908650',
  '00000908650',
  '00000908660',
  '00000908660',
  '00000908670',
 

In [16]:
set([1])

{1}

In [137]:
resp.request.url,resp.json

('http://test1.thirdpart.kaolatest.netease.com/api/queryAllGoodsId?channelId=1200&timestamp=2018-03-19+15%3A39%3A51&v=1.0&sign_method=md5&app_key=bb0b3ad64c9e5eb06c2fb6f163bf179e79051bd5c9b652fc45dc68a2b5dd23c6&sign=345944BD6BFD76EC5373BCB491D39F46',
 <bound method Response.json of <Response [200]>>)

In [ ]:
resp

In [5]:
# 单个商品详情查询
def query_goods_info_by_id(sku_id):
    """渠道下单个商品信息详情查询，接口地址：queryGoodsInfoById"""
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "skuId":sku_id,
        "queryType":0, # 0表示返回全部信息、1表示只返回关键信息如：商品id、价格和库存
        "channelSalePrice":'1' # 回传税后价，仅支持保税、直邮、一般贸易（税率为0）模式、海淘模式；个人物品清关模式不返回
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryGoodsInfoById',params=body)
    try:
        resp_data = resp.json()
    except:
        resp_data = resp.text
    return resp_data
query_goods_info_by_id('00001983401')

{'goodsInfo': {'IsPostageFree': False,
  'afterTaxPrice': '1.12',
  'brandCountryName': '韩国',
  'brandId': 1106,
  'brandName': 'MEDIHEAL 美迪恵尔',
  'canReturnGoods': 1,
  'category': '面膜/面膜粉 面部护肤 美容护肤 美容彩妆',
  'detail': '<p style="text-align:center;"><img src="http://haitao.nos.netease.com/ce8008ff17c8458a9401980d37858eac1510319102434j9twzxsx14730.jpg" /></p> \n<p style="text-align:center;"><img src="http://haitao.nos.netease.com/eae95bb491e54570ae9725c2021682e11520238856102jedyyopy10611.jpg?imageView&amp;quality=98&amp;crop=0_0_750_500" /></p> \n<p style="text-align:center;"><img src="http://haitao.nos.netease.com/eae95bb491e54570ae9725c2021682e11520238856102jedyyopy10611.jpg?imageView&amp;quality=98&amp;crop=0_500_750_69" /></p> \n<p style="text-align:center;"><img src="http://haitao.nos.netease.com/1bl7kah9k26_750_7373.jpg?imageView&amp;quality=98&amp;crop=0_1000_750_500" /></p> \n<p style="text-align:center;"><img src="http://haitao.nos.netease.com/1bl7kah9k26_750_7373.jpg?imageView

In [169]:
# 商品详情批量查询
def query_goods_info_by_ids(sku_ids):
    """商品信息详情批量查询，接口地址：queryGoodsInfoByIds 由于http请求对响应时间有限制，现在默认一次只能查询20条，多于20条会返回错误码-103，详情见错误码说明"""
    body = {
        "channelId":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "skuIds":str(sku_ids),
        "queryType":0, # 0表示返回全部信息、1表示只返回关键信息如：商品id、价格和库存
        "channelSalePrices":str([1]*len(sku_ids)) # 回传税后价，仅支持保税、直邮、一般贸易（税率为0）模式、海淘模式；个人物品清关模式不返回
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryGoodsInfoByIds',params=body)
    return resp.json()
query_goods_info_by_ids(['59070208-96359ea7676df2eea9ce9acf7c538005', '59076218-6f04aaa51ddfbe2aad2e5d139528bc95', '59077671-cdfe59eb911f5b727240c310df9fad57', '59077671-cdfe59eb911f5b727240c310df9fad57', '59083052-6f04aaa51ddfbe2aad2e5d139528bc95', '59086857-6f04aaa51ddfbe2aad2e5d139528bc95', '6014-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6024-d46389875f39726d27af6c3e81fdcf77', '6024-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6064-128010f98e698bb5365cc41a972e4b9f', '6626-bba69a3033db10d7908615a3dedeb6c5', '6720750-ecc4090b639c47f89b453980923afb8e', '6886-ecc4090b639c47f89b453980923afb8e', '7030302-ecc4090b639c47f89b453980923afb8e', '8979-3845321bfcf28ddc40b4cca6613834a3', '9017-fd619f0359020094b63a73656691297d', '9100-ecc4090b639c47f89b453980923afb8e'])

[{'goodsInfo': {'afterTaxPrice': '1.00',
   'brandCountryName': '美国',
   'brandId': 1008,
   'brandName': '三星111',
   'category': '直冲 战网 游戏点卡 虚拟商品',
   'detail': '123',
   'goodsId': 59070208,
   'goodsImages': [{'goodsId': 0,
     'imageType': 1,
     'imageUrl': 'http://pop.nosdn.127.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor430': 'http://pop.nosdn.127_430_430.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor640490': 'http://pop.nosdn.127_640_490.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'imageUrlFor70': 'http://pop.nosdn.127_70_70.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
     'orderValue': 1}],
   'goodsProperty': {},
   'imageUrl': 'http://pop.nosdn.127.net/02d2cb52-3a05-4f36-8f3d-10dabd524e8d',
   'importType': 3,
   'isFreeShipping': 0,
   'isFreeTax': 0,
   'isPresell': 0,
   'leafCategoryId': 24656,
   'logisticsProperty': None,
   'marketPrice': 388,
   'memberCount': 0,
   'onlineStatus': 0,
   'price': 388.0,
   'productId': 59070208,
   

In [146]:
sku_ids = ['59070208-96359ea7676df2eea9ce9acf7c538005', '59076218-6f04aaa51ddfbe2aad2e5d139528bc95', '59077671-cdfe59eb911f5b727240c310df9fad57', '59077671-cdfe59eb911f5b727240c310df9fad57', '59083052-6f04aaa51ddfbe2aad2e5d139528bc95', '59086857-6f04aaa51ddfbe2aad2e5d139528bc95', '6014-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6024-d46389875f39726d27af6c3e81fdcf77', '6024-e3d4f8c9df8d3ddf344d1d4ea1603b4a', '6064-128010f98e698bb5365cc41a972e4b9f', '6626-bba69a3033db10d7908615a3dedeb6c5', '6720750-ecc4090b639c47f89b453980923afb8e', '6886-ecc4090b639c47f89b453980923afb8e', '7030302-ecc4090b639c47f89b453980923afb8e', '8979-3845321bfcf28ddc40b4cca6613834a3', '9017-fd619f0359020094b63a73656691297d', '9100-ecc4090b639c47f89b453980923afb8e']

In [155]:
sku_ids[5]

'59086857-6f04aaa51ddfbe2aad2e5d139528bc95'

In [133]:
# 商品信息增量更新接口（性能待优化）
def query_changed_goods_info():
    """根据时间戳的商品信息更新接口 queryChangedGoodsInfo;该接口限制只能查询30天之内的数据;"""
    body = {
        "source":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "startTime":int(time.time())*1000,
        "endTime":int(time.time()+86400*24)*1000
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryChangedGoodsInfo',params=body)
    return resp.json()
query_changed_goods_info()

{'recCode': 200, 'recMeg': '查询成功', 'result': []}

In [170]:
def query_comment_info():
    """查询goodsId下面指定时间段内所有的评论信息；默认查询间隔不得大于三个月(90天);接口地址queryCommentInfo;"""
    body = {
        "source":channel_id,
        "timestamp":time.strftime("%Y-%m-%d %H:%M:%S"),
        "v":"1.0",
        "sign_method":"md5",
        "app_key":app_key,
        "sign":'',
        "startTime":int(time.time())*1000,
        "endTime":int(time.time()+86400*24)*1000,
        "goodsId":59070208
    }
    body['sign'] = gen_sign(body,secret_key)
    resp = requests.post(url+'queryChangedGoodsInfo',params=body)
    return resp.json()
query_comment_info()

{'recCode': 200, 'recMeg': '查询成功', 'result': []}

In [194]:
# 一、获取分类categoryId 和  title
category_list = []
brand_list = []

for item in requests.get('https://sp.kaola.com/api/category?V330').json()['body']['categoryTreeMenuList']:
    if 'categoryId' not in item:
        continue
    resp = requests.get(f'https://sp.kaola.com/api/category/frontCategory?V330&categoryId={item["categoryId"]}').json()['body']
    # 'brandList', 'level2CategoryList'
    category_tree = {'categoryId': item['categoryId'], 'categoryName': item['title'],'childCategoryViewList':resp['level2CategoryList']}
    pprint(item)
    category_list.append(category_tree)
with open('kaola_category_tree.json','w') as f:
    f.write(json.dumps(category_list,indent=2,ensure_ascii=False))

{'categoryId': 8171, 'title': '美容彩妆', 'type': 2}
{'categoryId': 8181, 'title': '奶粉纸尿裤', 'type': 2}
{'categoryId': 8182, 'title': '母婴专区', 'type': 2}
{'categoryId': 8183, 'title': '轻奢', 'type': 2}
{'categoryId': 8185, 'title': '营养保健', 'type': 2}
{'categoryId': 8184, 'title': '个人洗护', 'type': 2}
{'categoryId': 8187, 'title': '手表配饰', 'type': 2}
{'categoryId': 8188, 'title': '数码家电', 'type': 2}
{'categoryId': 8190, 'title': '运动户外', 'type': 2}
{'categoryId': 15413, 'title': '家居生活', 'type': 2}
{'categoryId': 8189, 'title': '环球美食', 'type': 2}
{'categoryId': 8186, 'title': '服饰鞋靴', 'type': 2}
{'categoryId': 17236, 'title': '全球工厂店', 'type': 2}
{'categoryId': 15424, 'title': '网易严选', 'type': 2}
{'categoryId': 8191, 'title': '生鲜', 'type': 2}
{'categoryId': 14757, 'title': '宠物生活', 'type': 2}
{'categoryId': 14048, 'title': '考拉超市', 'type': 2}


In [221]:
import tablib
url = "http://dpp.boluome.com/https/generate"
tab_data = tablib.Dataset(headers=['一级分类','二级分类','三级分类','iconUrl'])
for category_1 in category_list:
    for category_2 in category_1['childCategoryViewList']:
        for category_3 in category_2['childCategoryViewList']:
            icon = category_3['iconUrl']
            os.system("wget {} -P /data/kaola/ -q".format(icon))
            new_icon = 'https://f1.otosaas.com/img/icon/kaola/'+ icon.split('/')[-1]
            response = requests.post(url,data={"https": new_icon},
                                     files={"image": open(f"/data/kaola/{icon.split('/')[-1]}", 'rb')}).json()

            tab_data.append([category_1['categoryName'],category_2['categoryName'],category_3['categoryName'],response['https']])
with open('kaola_category_tree.xlsx','wb') as f:
    f.write(tab_data.xlsx)

In [8]:
goods_info = {
        "goodsId": 28539, 
        "detail": """<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_0_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_1000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_1500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_2000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_2500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_3000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_3500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_4000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_4500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_5000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_5500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_6000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_6500_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_7000_750_500\" /></p>
<p style=\"text-align:center;\"><img src=\"http://haitao.nos.netease.com/ff8b608f24c44dd592b3c1dc405752061515046612878jc03mw7i10358.jpg?imageView&quality=98&crop=0_7500_750_291\" /></p>""", 
        "store": 9007, 
        "imageUrl": "http://haitao.nos.netease.com/onlineia666tz711682.jpg", 
        "warehouseStores": {
            "深圳前海保税仓": {
                "warehouseName": "深圳前海保税仓", 
                "warehouseId": 2153, 
                "warehouseStore": 0
            }, 
            "杭州网仓仓库": {
                "warehouseName": "杭州网仓仓库", 
                "warehouseId": 13, 
                "warehouseStore": 9007
            }, 
            "宁波顺丰保税仓": {
                "warehouseName": "宁波顺丰保税仓", 
                "warehouseId": 6, 
                "warehouseStore": 0
            }, 
            "天津网仓保税仓": {
                "warehouseName": "天津网仓保税仓", 
                "warehouseId": 3554, 
                "warehouseStore": 0
            }, 
            "郑州保税仓": {
                "warehouseName": "郑州保税仓", 
                "warehouseId": 7, 
                "warehouseStore": 0
            }, 
            "杭州海仓科技有限公司东芝仓库": {
                "warehouseName": "杭州海仓科技有限公司东芝仓库", 
                "warehouseId": 46, 
                "warehouseStore": 0
            }, 
            "广州南沙保税仓": {
                "warehouseName": "广州南沙保税仓", 
                "warehouseId": 9001, 
                "warehouseStore": 0
            }, 
            "网易考拉杭州来福士直提店": {
                "warehouseName": "网易考拉杭州来福士直提店", 
                "warehouseId": 9281, 
                "warehouseStore": 0
            }, 
            "网易6号仓": {
                "warehouseName": "网易6号仓", 
                "warehouseId": 5, 
                "warehouseStore": 0
            }, 
            "重庆保税仓": {
                "warehouseName": "重庆保税仓", 
                "warehouseId": 3006, 
                "warehouseStore": 0
            }
        }, 
        "isFreeShipping": 1, 
        "onlineStatus": 1, 
        "brandId": 2052, 
        "brandName": "Aptamil 爱他美（澳洲）", 
        "subTitle": "爱他美白金版奶粉，在促进婴儿免疫系统、大脑发育和新陈代谢方面比爱他美普通版系列更优。而且奶粉无添加香料，不会造成宝宝口味的依赖。", 
        "recommandStore": {
            "warehouseName": "杭州网仓仓库", 
            "warehouseId": 13, 
            "warehouseStore": 9007
        }, 
        "title": "Aptamil 澳洲爱他美 Profutura 白金版婴儿奶粉 1段 900克/罐 2罐装", 
        "IsPostageFree": false, 
        "suggestPrice": 396, 
        "thirdCategoryName": "婴幼儿奶粉", 
        "logisticsProperty": null, 
        "skuId": "36938-68a3e5516d7a7dc21fbe0e7ee13bfc1c", 
        "goodsProperty": {
            "奶粉种类": [
                "牛奶粉"
            ], 
            "产地": [
                "新西兰"
            ], 
            "净含量": [
                "1-2千克"
            ], 
            "包装": [
                "罐装", 
                "组合装"
            ], 
            "存储方式": [
                "干燥通风处保存"
            ], 
            "适用阶段": [
                "一段"
            ], 
            "版本": [
                "澳洲白金版"
            ]
        }, 
        "isSelf": true, 
        "leafCategoryId": 626, 
        "isPresell": 0, 
        "marketPrice": 398, 
        "taxRate": 0.119, 
        "canReturnGoods": 0, 
        "taxRates": [
            0.119
        ], 
        "memberCount": 2, 
        "productId": 36938, 
        "brandCountryName": "新西兰", 
        "importType": 1, 
        "category": "婴幼儿牛奶粉 婴幼儿牛奶粉 婴幼儿奶粉 母婴", 
        "price": XXXXXX, 
        "skuProperty": [ ], 
        "shortTitle": "澳洲爱他美Profutura 白金版婴儿奶粉1段900克/罐2罐", 
        "goodsImages": [
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nos.netease.com/onlineia666tz711682.jpg", 
                "imageType": 1, 
                "orderValue": 1, 
                "imageUrlFor70": "http://haitao.nos.netease.com/onlineia666tz711682_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nos.netease.com/onlineia666tz711682_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nos.netease.com/onlineia666tz711682_640_490.jpg"
            }, 
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nosdn5.127.net/iofbjdv011_800_800.jpg", 
                "imageType": 1, 
                "orderValue": 2, 
                "imageUrlFor70": "http://haitao.nosdn5.127.net/iofbjdv011_800_800_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nosdn5.127.net/iofbjdv011_800_800_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nosdn5.127.net/iofbjdv011_800_800_640_490.jpg"
            }, 
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nosdn2.127.net/iofbjekd54_800_800.jpg", 
                "imageType": 1, 
                "orderValue": 3, 
                "imageUrlFor70": "http://haitao.nosdn2.127.net/iofbjekd54_800_800_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nosdn2.127.net/iofbjekd54_800_800_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nosdn2.127.net/iofbjekd54_800_800_640_490.jpg"
            }, 
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nos.netease.com/iofbje2c82_800_800.jpg", 
                "imageType": 1, 
                "orderValue": 4, 
                "imageUrlFor70": "http://haitao.nos.netease.com/iofbje2c82_800_800_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nos.netease.com/iofbje2c82_800_800_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nos.netease.com/iofbje2c82_800_800_640_490.jpg"
            }, 
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nosdn5.127.net/iofbje7p64_800_800.jpg", 
                "imageType": 1, 
                "orderValue": 5, 
                "imageUrlFor70": "http://haitao.nosdn5.127.net/iofbje7p64_800_800_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nosdn5.127.net/iofbje7p64_800_800_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nosdn5.127.net/iofbje7p64_800_800_640_490.jpg"
            }, 
            {
                "goodsId": 0, 
                "imageUrl": "http://haitao.nosdn3.127.net/iofbjec728_800_800.jpg", 
                "imageType": 1, 
                "orderValue": 6, 
                "imageUrlFor70": "http://haitao.nosdn3.127.net/iofbjec728_800_800_70_70.jpg", 
                "imageUrlFor430": "http://haitao.nosdn3.127.net/iofbjec728_800_800_430_430.jpg", 
                "imageUrlFor640490": "http://haitao.nosdn3.127.net/iofbjec728_800_800_640_490.jpg"
            }
        ], 
        "storage": "杭州网仓仓库", 
        "isFreeTax": 1, 
        "thirdCategoryId": 4891
    }

NameError: name 'false' is not defined

In [6]:
from connection import ConnectionTask
import ast
c = ConnectionTask()
rdb = c.dpprdb(0)

In [11]:
old_goods_id_set = rdb.get('dpp:shopping:kaola:goodsIdSet')
old_goods_id_set = ast.literal_eval(old_goods_id_set) if old_goods_id_set else set()
new_goods_id_set = set([1767104])
remove_goods_id_set = old_goods_id_set - new_goods_id_set
print(remove_goods_id_set)

{'1911228', '1955902', '151570', '1873619', '1542849', '1392631', '1628887', '158927', '1545041', '158526', '1374060', '1485275', '157143', '1266766', '1310720', '276258', '18548', '249251', '00002331011', '1767104', '1858570', '00005643781', '1980849', '1949407', '1385856', '1547587', '1428642', '1614562', '9246', '1313421', '1386933', '1400105', '10627', '1496088', '157604', '17601', '157955', '1075357', '1473561', '1577666', '1622930', '1456387', '1505827', '1383085', '36775', '6726', '157891', '1829841', '1452403', '36908', '158397', '1565839', '1790680', '273453', '1998276', '1310700', '1644860', '1978798', '1805054', '1969261', '1573035', '1316021', '1450542', '1232156', '1343063', '1566872', '158907', '279929', '2023454', '00000908670', '1233980', '1877574', '1565003', '1498034', '00002172091', '10589', '22456', '1352044', '5546', '1959241', '1648229', '1812149', '13884', '1871794', '295723', '158008', '29932', '1296706', '1838170', '1371958', '157838', '1310692', '1477182', '18

In [ ]:



self.rdb.set('dpp:shopping:kaola:goodsIdSet', new_goods_id_set)